# Parameters

In [1]:
batch_size     = 8
learning_rate  = 0.0001
epochs         = 7

# Define model

In [2]:
from model import DepthEstimate

model = DepthEstimate()

Base model loaded DenseNet169

Model created.


# Data loader

In [3]:
from data import DataLoader

dl = DataLoader()
train_generator = dl.get_batched_dataset(batch_size)

# Get the validation dataset
validation_dataset = dl.get_validation_dataset(batch_size)

print('Data loader ready.')

nyu2_train :  [['data/nyu2_train/living_room_0038_out/37.jpg', 'data/nyu2_train/living_room_0038_out/37.png'], ['data/nyu2_train/living_room_0038_out/115.jpg', 'data/nyu2_train/living_room_0038_out/115.png'], ['data/nyu2_train/living_room_0038_out/6.jpg', 'data/nyu2_train/living_room_0038_out/6.png'], ['data/nyu2_train/living_room_0038_out/49.jpg', 'data/nyu2_train/living_room_0038_out/49.png'], ['data/nyu2_train/living_room_0038_out/152.jpg', 'data/nyu2_train/living_room_0038_out/152.png'], ['data/nyu2_train/living_room_0038_out/142.jpg', 'data/nyu2_train/living_room_0038_out/142.png'], ['data/nyu2_train/living_room_0038_out/94.jpg', 'data/nyu2_train/living_room_0038_out/94.png'], ['data/nyu2_train/living_room_0038_out/158.jpg', 'data/nyu2_train/living_room_0038_out/158.png'], ['data/nyu2_train/living_room_0038_out/18.jpg', 'data/nyu2_train/living_room_0038_out/18.png'], ['data/nyu2_train/living_room_0038_out/167.jpg', 'data/nyu2_train/living_room_0038_out/167.png'], ['data/nyu2_train

# Compile & Train

In [4]:
import tensorflow
from loss import depth_loss_function

optimizer = tensorflow.keras.optimizers.legacy.Adam(lr=learning_rate, amsgrad=True)

# model.compile(loss=depth_loss_function, optimizer=optimizer)

/Users/adeelhafeez/GPU_Test/env/lib/python3.10/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [5]:
# Create checkpoint callback
import os
checkpoint_path = "training_11/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tensorflow.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, save_best_only=True, verbose=1)


In [7]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from itertools import product
import random

# Define hyperparameters
batch_size = 8
learning_rate = 0.0001
epochs = 7  # Reduce the number of epochs for faster training

# Define weight range for random search
weight_range = [0.20, 0.40, 0.60, 0.80, 1.0]

# Number of random combinations to try
num_random_combinations = 20

# Initialize the DataLoader
dl = DataLoader(DEBUG=False)

# Create a list to store loss values
val_loss_values = []

# Placeholder for best validation loss and corresponding weights
best_val_loss = float('inf')
best_weights = None

# Perform random search
for _ in range(num_random_combinations):
    # Randomly sample weights from the weight range
    w1 = random.choice(weight_range)
    w2 = random.choice(weight_range)
    w3 = random.choice(weight_range)

    # Create a new instance of the model for each combination of weights
    model = DepthEstimate()  # Initialize the model

    # Compile the model with the current weights
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    def custom_loss(y_true, y_pred):
        global w1, w2, w3  # Add these lines to access the global weights
        return depth_loss_function(y_true, y_pred, maxDepthVal=1000.0/10.0)

    model.compile(loss=custom_loss, optimizer=optimizer)

    # Create training and validation datasets using your DataLoader class
    train_dataset = dl.get_batched_dataset(batch_size)
    validation_dataset = dl.get_validation_dataset(batch_size)

    # Calculate the number of steps per epoch based on the dataset size and batch size
    steps_per_epoch = dl.length // batch_size

    # Create a checkpoint callback for saving model weights
    checkpoint_path = f"training_weights_w1_{w1}_w2_{w2}_w3_{w3}/cp.ckpt"
    cp_callback = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path,
        save_weights_only=True,
        save_best_only=True,
        verbose=1
    )

    # Inside your training loop
    history = model.fit(
        train_dataset,
        epochs=epochs,  # Train for the specified number of epochs
        steps_per_epoch=steps_per_epoch,  # Specify the number of steps
        verbose=1,
        callbacks=[cp_callback]
    )

    # Calculate the average validation loss after training
    val_loss = model.evaluate(validation_dataset, verbose=0)
    val_loss_values.append(val_loss)

    # Check if this combination of weights results in a lower validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_weights = (w1, w2, w3)

# Print the best weights for this combination
print(f"Best Weights (w1, w2, w3): {best_weights}")
print(f"Best Validation Loss: {best_val_loss}")


nyu2_train :  [['data/nyu2_train/living_room_0038_out/37.jpg', 'data/nyu2_train/living_room_0038_out/37.png'], ['data/nyu2_train/living_room_0038_out/115.jpg', 'data/nyu2_train/living_room_0038_out/115.png'], ['data/nyu2_train/living_room_0038_out/6.jpg', 'data/nyu2_train/living_room_0038_out/6.png'], ['data/nyu2_train/living_room_0038_out/49.jpg', 'data/nyu2_train/living_room_0038_out/49.png'], ['data/nyu2_train/living_room_0038_out/152.jpg', 'data/nyu2_train/living_room_0038_out/152.png'], ['data/nyu2_train/living_room_0038_out/142.jpg', 'data/nyu2_train/living_room_0038_out/142.png'], ['data/nyu2_train/living_room_0038_out/94.jpg', 'data/nyu2_train/living_room_0038_out/94.png'], ['data/nyu2_train/living_room_0038_out/158.jpg', 'data/nyu2_train/living_room_0038_out/158.png'], ['data/nyu2_train/living_room_0038_out/18.jpg', 'data/nyu2_train/living_room_0038_out/18.png'], ['data/nyu2_train/living_room_0038_out/167.jpg', 'data/nyu2_train/living_room_0038_out/167.png'], ['data/nyu2_train

Base model loaded DenseNet169

Model created.
Epoch 1/7
71/71 [==============================] - ETA: 0s - loss: 1.1191WARNING:tensorflow:Can save best model only with val_loss available, skipping.


71/71 [==============================] - 136s 2s/step - loss: 1.1191
Epoch 2/7
71/71 [==============================] - ETA: 0s - loss: 0.7643WARNING:tensorflow:Can save best model only with val_loss available, skipping.


71/71 [==============================] - 127s 2s/step - loss: 0.7643
Epoch 3/7
71/71 [==============================] - ETA: 0s - loss: 0.7288WARNING:tensorflow:Can save best model only with val_loss available, skipping.


71/71 [==============================] - 125s 2s/step - loss: 0.7288
Epoch 4/7
71/71 [==============================] - ETA: 0s - loss: 0.5394WARNING:tensorflow:Can save best model only with val_loss available, skipping.


...

71/71 [==============================] - 147s 2s/step - loss: 0.6972
Epoch 4/7
71/71 [==============================] - ETA: 0s - loss: 0.6395WARNING:tensorflow:Can save best model only with val_loss available, skipping.


71/71 [==============================] - 147s 2s/step - loss: 0.6395
Epoch 5/7
71/71 [==============================] - ETA: 0s - loss: 0.5570WARNING:tensorflow:Can save best model only with val_loss available, skipping.


71/71 [==============================] - 148s 2s/step - loss: 0.5570
Epoch 6/7
71/71 [==============================] - ETA: 0s - loss: 0.4836WARNING:tensorflow:Can save best model only with val_loss available, skipping.


71/71 [==============================] - 146s 2s/step - loss: 0.4836
Epoch 7/7
71/71 [==============================] - ETA: 0s - loss: 0.4368WARNING:tensorflow:Can save best model only with val_loss available, skipping.


71/71 [==============================] - 146s 2s/step - loss: 0.4368
Best Weights (w1, w2, w3): (1.0, 0.2, 0.6)
Best Validation Loss: 0.349799245595932


In [ ]:
print("Best Weights (w1, w2, w3):", best_weights)
print("Best Validation Loss:", best_val_loss)